In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("joebeachcapital/hotel-reviews")

print("Path to dataset files:", path)

/Users/oceanspiess/PycharmProjects/ML_NLP/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/oceanspiess/PycharmProjects/ML_NLP/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/oceanspiess/.cache/kagglehub/datasets/joebeachcapital/hotel-reviews/versions/2


In [7]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords') 
nltk.download('wordnet')

# Load the dataset
df = pd.read_csv(f"{path}/reviews.csv")

df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/oceanspiess/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oceanspiess/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oceanspiess/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,ratings,title,text,author,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile
0,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...","“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,"{'username': 'Papa_Panda', 'num_cities': 22, '...",December 2012,93338,0,2012-12-17,147643103,False
1,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...","{'username': 'Maureen V', 'num_reviews': 2, 'n...",December 2012,93338,0,2012-12-17,147639004,False
2,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Great Stay”,This is a great property in Midtown. We two di...,"{'username': 'vuguru', 'num_cities': 12, 'num_...",December 2012,1762573,0,2012-12-18,147697954,False
3,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,"{'username': 'Hotel-Designer', 'num_cities': 5...",August 2012,1762573,0,2012-12-17,147625723,False
4,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,"{'username': 'JamesE339', 'num_cities': 34, 'n...",December 2012,1762573,0,2012-12-17,147612823,False


In [8]:
# Extract ratings dictionary into separate columns
df['ratings'] = df['ratings'].apply(eval)  # Convert string to dictionary
rating_cols = ['service', 'cleanliness', 'value', 'sleep_quality', 'rooms', 'location', 'overall']

for col in rating_cols:
    df[col] = df['ratings'].apply(lambda x: x.get(col))

# Drop unnecessary columns and ratings dictionary
columns_to_drop = ['title', 'author', 'date_stayed', 'num_helpful_votes', 
                  'date', 'id', 'via_mobile', 'ratings']
df = df.drop(columns=columns_to_drop)

# Reorder columns to put offering_id first
cols = ['offering_id'] + [col for col in df.columns if col != 'offering_id']
df = df[cols]

# Display first few rows of cleaned dataframe
df.head()



,offering_id,text,service,cleanliness,value,sleep_quality,rooms,location,overall
0,93338,Stayed in a king suite for 11 nights and yes i...,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1,93338,"On every visit to NYC, the Hotel Beacon is the...",5.0,5.0,5.0,5.0,5.0,5.0,5.0
2,1762573,This is a great property in Midtown. We two di...,4.0,5.0,4.0,4.0,4.0,5.0,4.0
3,1762573,The Andaz is a nice hotel in a central locatio...,5.0,5.0,5.0,5.0,5.0,5.0,4.0
4,1762573,I have stayed at each of the US Andaz properti...,4.0,5.0,3.0,5.0,5.0,5.0,4.0


In [ ]:
# Group texts by offering_id and concatenate them
df = df.groupby('offering_id').agg({
    'text': ' '.join,
    'service': 'mean',
    'cleanliness': 'mean', 
    'value': 'mean',
    'sleep_quality': 'mean',
    'rooms': 'mean',
    'location': 'mean',
    'overall': 'mean'
}).reset_index()



In [12]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords') 
nltk.download('wordnet')

# Initialize lemmatizer and get stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    """
    Clean text by:
    1. Converting to lowercase
    2. Removing special characters and punctuation (keeping newlines)
    3. Removing stopwords
    4. Lemmatizing words
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and punctuation but keep newlines
    text = re.sub(r'[^\w\s\n]', ' ', text)
    
    # Tokenize
    words = word_tokenize(text)
    
    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and not word.isnumeric()]
    
    # Join words back together, preserving newlines
    cleaned_text = ' '.join(words)
    
    return cleaned_text

# Apply text cleaning to the text column
df['text'] = df['text'].apply(clean_text)

# Display first few rows to verify cleaning
print("First few cleaned texts:")
print(df['text'].head())


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/oceanspiess/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oceanspiess/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oceanspiess/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


First few cleaned texts:
0    make fast visit seattle found pioneer regret h...
1    great service room clean could use queen bed t...
2    beautiful view space needle especially night l...
3    hotel need serious update room big bed comfort...
4    experience day inn perfect staff great manager...
Name: text, dtype: object


In [ ]:
# Install and import Rank-BM25
# !pip install rank_bm25
from rank_bm25 import BM25Okapi

# Convert cleaned texts to tokenized documents for BM25
tokenized_corpus = [text.split() for text in df['text']]

# Create BM25 object
bm25 = BM25Okapi(tokenized_corpus)

# Function to perform BM25 search
def bm25_search(query, top_k=5):
    """
    Search through corpus using BM25 ranking
    
    Args:
        query (str): Search query
        top_k (int): Number of top results to return
        
    Returns:
        list: Top k matching documents and their scores
    """
    # Clean and tokenize query using same preprocessing
    cleaned_query = clean_text(query)
    tokenized_query = cleaned_query.split()
    
    # Get document scores
    doc_scores = bm25.get_scores(tokenized_query)
    
    # Get top k documents
    top_n = np.argsort(doc_scores)[::-1][:top_k]
    
    results = []
    for idx in top_n:
        results.append({
            'index': idx,
            'score': doc_scores[idx],
            'text': df['text'].iloc[idx]
        })
        
    return results

# Example usage
print("\nExample BM25 search:")
example_query = "great hotel service"
results = bm25_search(example_query, top_k=3)
for i, result in enumerate(results, 1):
    print(f"\nResult {i}:")
    print(f"Score: {result['score']:.2f}")
    print(f"Text: {result['text'][:200]}...")
 